In [1]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px


In [3]:
# Load the spacex_launch_dash.csv file
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Print column names to check available columns
print("Available columns:", spacex_df.columns)


Available columns: Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')


In [4]:
# Create a dash application
app = dash.Dash(__name__)

In [5]:
# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True
    ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[0, 10000]
    ),

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


In [6]:
# TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', 
                     title='Total Success Launches By Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', 
                     title=f'Success vs Failure for site {entered_site}')
    return fig


In [8]:
# TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id="payload-slider", component_property="value")]
)
def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    
    if entered_site == 'ALL':
        fig = px.scatter(spacex_df[mask], x="Payload Mass (kg)", y="class", 
                         color="Booster Version Category",
                         title='Correlation between Payload and Success for all Sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df[mask], x="Payload Mass (kg)", y="class", 
                         color="Booster Version Category",
                         title=f'Correlation between Payload and Success for site {entered_site}')
    return fig


In [9]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

1. Which site has the largest successful launches?
Use the pie chart when "All Sites" is selected in the dropdown. The largest slice represents the site with the most successful launches.

2. Which site has the highest launch success rate?
Select each site individually in the dropdown and compare the success vs. failure ratio in the resulting pie charts.

3. Which payload range(s) has the highest launch success rate?
Use the payload range slider and observe the scatter plot. Look for ranges where most points are at the top (successful launches).

4. Which payload range(s) has the lowest launch success rate?
Similar to #3, but look for ranges where most points are at the bottom (failed launches).

5. Which F9 Booster version has the highest launch success rate?
In the scatter plot, observe the color-coding of points (representing booster versions). Look for the color that appears most frequently at the top of the chart.

